# WPP: total population

## Parameters

In [ ]:
dest_dir = '/tmp/wpp_2019_total_population'

## Walden

In [ ]:
from owid import walden

In [ ]:
walden_ds = walden.Catalog().find_one('wpp', '2019', 'standard_projections')

In [ ]:
walden_ds

## Unzip

In [ ]:
import tempfile
import shutil

In [ ]:
temp_dir = tempfile.mkdtemp()

In [ ]:
import zipfile

In [ ]:
zipfile.ZipFile(walden_ds.local_path).extractall(temp_dir)

In [ ]:
!ls {temp_dir}/WPP2019

## Make dataset

In [ ]:
from owid.catalog import Dataset

In [ ]:
ds = Dataset.create_empty(dest_dir)

## Add tables

In [ ]:
from owid.catalog import Table
import pandas as pd

### Total population

In [ ]:
df = pd.read_csv(f'{temp_dir}/WPP2019/WPP2019_TotalPopulationBySex.csv')

In [ ]:
df.head()

In [ ]:
df.columns = ['loc_id', 'location', 'var_id', 'variant', 'year', 'mid_period', 'population_male', 'population_female', 'population_total', 'population_density']

In [ ]:
t = Table(df[['loc_id', 'location']].drop_duplicates().set_index('loc_id'))
t.metadata.short_name = 'location_codes'
ds.add(t)

In [ ]:
t = Table(df[['var_id', 'variant']].drop_duplicates().set_index('var_id'))
t.metadata.short_name = 'variant_codes'
ds.add(t)

In [ ]:
df.drop(columns=['loc_id', 'var_id'], inplace=True)

In [ ]:
df.set_index(['variant', 'location', 'year'], inplace=True)

In [ ]:
df

In [ ]:
df.index.levels[0]

In [ ]:
t = Table(df)
t.metadata.short_name = 'total_population'
ds.add(t)

## Clean up

In [ ]:
shutil.rmtree(temp_dir)